```
SOW-MKI95 Computer Graphics & Computer Vision
Wave function collapse
```

```
Group number: ...
Student 1 name/number: ...
Student 2 name/number: ...
...
```

In [1]:
import random
import io
import base64
from collections import namedtuple
import numpy as np
from PIL import Image, ImageDraw
from IPython import display
import matplotlib.pyplot as plt


In [2]:
from enum import Enum, auto

class Direction(Enum):
    RIGHT = 0; UP = 1; LEFT = 2; DOWN = 3
    
    def reverse(self):
        return {Direction.RIGHT: Direction.LEFT,
                Direction.LEFT: Direction.RIGHT,
                Direction.UP: Direction.DOWN,
                Direction.DOWN: Direction.UP}[self]

In [ ]:
# %%
def neighbors(location, height, width):
    res = []
    x, y = location
    if x != 0:
        res.append((Direction.UP, x-1, y))
    if y != 0:
        res.append((Direction.LEFT, x, y-1))
    if x < height - 1:
        res.append((Direction.DOWN, x+1, y))
    if y < width - 1:
        res.append((Direction.RIGHT, x, y+1))
    return res

In [5]:
# %%
def find_true(array):
    """
    Like np.nonzero, except it makes sense.
    """
    transform = int if len(np.asarray(array).shape) == 1 else tuple
    return list(map(transform, np.transpose(np.nonzero(array))))

In [23]:
def propagate_bt(potential, start_location):

    height, width = potential.shape[:2]
    
    # Initialise grid of who needs an update; initially only the start loc
    needs_update = np.full((height, width), False)
    needs_update[start_location] = True
    
    while np.any(needs_update):
        # While there is still a location that needs an update, 
        # initialise a similar grid for the neighbors
        needs_update_next = np.full((height, width), False)

        # Get all the locations that need an update
        locations = find_true(needs_update)
        # for every space that is needs to be updated
        for location in locations:
            #find possible tiles
            
            possible_tiles = [unique_tiles[n] for n in find_true(potential[location])]
            #for every neighbor
            for neighbor in neighbors(location, height, width):
                neighbor_direction, neighbor_x, neighbor_y = neighbor
                neighbor_location = (neighbor_x, neighbor_y)
                
                # Add a constraint to the neighbor location; update the potential, keep going
                was_updated = add_constraint_bt(potential, neighbor_location,
                                             neighbor_direction, possible_tiles, current_tile=location)
                needs_update_next[neighbor_location] |= was_updated 
        needs_update = needs_update_next


In [7]:
def get_candidates(potential):
    '''
    Returns a list of all coordinates which can be collapsed in 
    order max constrained to min constrained 
    '''
    # 2d array where each entry is the number of choices
    num_choices = np.sum(potential, axis=2, dtype='float32')

    # if there's only one choice, set it to inf (one choice is no choice)
    num_choices[num_choices == 1] = np.inf
    
    if np.all(num_choices) == np.inf: 
        return None

    # get the indexes in order of the sums, in ascending order
    candidate_locations = np.unravel_index(np.argsort(num_choices, axis=None), num_choices.shape)
    candidate_locations = list(zip(*candidate_locations))
    return candidate_locations

In [27]:
def backtrack(potential, images):
    old_potential = potential.copy()
    # display.display(show_state(potential, tiles))
    display.display(show_state(potential, unique_tiles))
    images.append(show_state(potential, unique_tiles))

    # Done
    if done(potential):
        print('It is done.')
        out = io.BytesIO()
        images[0].save(out, format='gif', save_all=True, append_images=images[1:],
              duration=50, loop=0)
        return True
    else:
        # Otherwise get all candidate locations that still need to be decided
        candidates = get_candidates(potential)
        for candidate in candidates:
                ### Prepare

            # find possible tiles for this canditate 
            p_tiles = find_true(potential[candidate])

            # choose a random one according to weights
            if len(p_tiles) > 0:              
                tile_probs = weights[p_tiles] / sum(weights[p_tiles])
                # sort files randomly according to frequency of tile
                p_tiles = np.random.choice(p_tiles, p=tile_probs, size=len(p_tiles), replace=False)

            for tile in p_tiles:
                potential[candidate] = False
                potential[candidate][tile] = True 
                propagate_bt(potential, candidate)

                # if not finsihed after propagate 
                if np.any(potential[candidate]):
                    
                    ### Recurse

                    solved = backtrack(potential, images)
                    if solved:
                        return True

                ### Repair

                potential = old_potential


def done(potential):
    return np.sum(potential) == potential.shape[0] * potential.shape[1]



In [45]:
## change this so the potential tiles that do not overlapp are deleted from the potential list

def add_constraint_bt(potential, location, incoming_direction, possible_tiles, current_tile):
    changed = False
    
 
    # für alle tiles
    # potential[location] is the neighbor 
    for i_p, p in enumerate(potential[location]):
        if not p:
            continue
# if tile does not overlap with neighbors, set it to false 
# if coming from links, see if last two and first two coloumns overlapp
#if coming form ... 
        # if the direction is down 
        if incoming_direction == Direction.DOWN:
            # check for every tile that can be placed 
            for potential_tile in possible_tiles: 

                if not(potential_tile[0:2] == unique_tiles[i_p][1:3]).all():
                    potential[location][i_p] = False
                    changed = True

   
        if incoming_direction == Direction.UP:
          for potential_tile in possible_tiles: 
        
              if not (potential_tile [1:3] == unique_tiles[i_p][0:2]).all():
                  potential[location][i_p] = False  
                  changed = True


        if incoming_direction == Direction.LEFT:
          for potential_tile in possible_tiles:      
           
              if not (potential_tile[:,1:3] == unique_tiles[i_p][:,0:2]).all():
                  potential[location][i_p] = False 
                  changed = True



        if incoming_direction == Direction.RIGHT:
          for potential_tile in possible_tiles: 
           
              if not (potential_tile[:,0:2] == unique_tiles[i_p][:,1:3]).all():
                  potential[location][i_p] = False 
                  changed = True

    return changed

###Create Tiles from Image

In [9]:

def tile_image(bitmap, n=3):
    
   
    tiles =np.empty((bitmap.shape[0]-(n-1),bitmap.shape[1]-(n-1), 3, 3, 3)) 
    for x in range(bitmap.shape[0]-(n-1)):
      for y in range(bitmap.shape[1]-(n-1)):
        tiles[x,y] = bitmap[x:x+n, y:y+n, :]
    return tiles

    #tile_grid = np.zeros(bitmap.shape + (n,n))
    #bm_width, bm_height, _ = bitmap.shape
    #padded = np.pad(bitmap, ((0, n-1), (0, n-1), (0,0)), mode='wrap')
    #for x in range(bm_width):
     #   for y in range(bm_height):
     #       tile_grid[x, y] = padded[x:x+n, y:y+n, :]



def get_unique_tiles(tile_grid):
  all_tiles = np.empty((tile_grid.shape[0],tile_grid.shape[1],6,3,3,3))
  for row_idx, row in enumerate(tile_grid):
    for tile_idx, tile in enumerate(row): 
      all_tiles[row_idx][tile_idx][0]= tile
      all_tiles[row_idx][tile_idx][1] =np.rot90(tile)
      all_tiles[row_idx][tile_idx][2] =np.rot90(tile,2)
      all_tiles[row_idx][tile_idx][3] =np.rot90(tile,3)
      all_tiles[row_idx][tile_idx][4] =np.fliplr(tile)
      all_tiles[row_idx][tile_idx][5] =np.flipud(tile)

  all_tiles = all_tiles.reshape(tile_grid.shape[0]*tile_grid.shape[1]*6,3,3,3) 
  unique_tiles, counts_elements = np.unique(all_tiles, return_counts = True, axis=0)

  return unique_tiles, counts_elements


In [51]:

image = Image.open('/content/City.png')
display.display(image)
bitmap = np.asarray(image)
bitmap = bitmap[:,:,:3]
tile_grid = tile_image(bitmap)
unique_tiles, weights = get_unique_tiles(tile_grid)
print("possible Tiles")
for tile in unique_tiles:
  image = Image.fromarray(tile.astype(np.uint8))
  display.display(image)
print(len(unique_tiles))


possible Tiles


25


In [49]:
# output image pixel size 
output_length = 25 
output_height = 25

In [12]:
def blend_tiles(choices, tiles):
  # gives back average RGB value of the top left corner 
  if len(find_true(choices)) == 0:
    return np.asarray([200,200,200])
  wert = np.asarray([tiles[i,0,0] for i in range(len(choices)) if choices[i]])
  wert = np.average(wert, axis=0)

  return wert


def show_state(potential, tiles):
    rows = []
    # für jede reihe im bild
    for row in potential:
      row_pixels = []
      for t in row: 
        pixel = blend_tiles(t, tiles)
        row_pixels.append(pixel)

      rows.append(row_pixels)
 
    rows = np.array(rows)

    image = rows.reshape(output_length , output_height, 3)  
    image = Image.fromarray(image.astype(np.uint8))
    return image


In [ ]:
# create list which keeps track which tiles are still possible 
potential = np.full((output_length, output_height, len(unique_tiles)), True)
# create output image bitmap
output = np.empty((output_length, output_height,3))

display.display(show_state(potential, unique_tiles))
images = [show_state(potential, unique_tiles)]
backtrack(potential, images)

# %%
out = io.BytesIO()
images[0].save(out, format='gif', save_all=True, append_images=images[1:],
               duration=50, loop=0)
images[-1]

# %%
import base64
from IPython import display
display.HTML('<img src="data:image/gif;base64,{0}">'
             .format(base64.b64encode(out.getvalue()).decode('utf8')))